## Matt Sloane / ms9548

#### Homework 9 - Assignment 1 - Inclass Lab

In [16]:
# Import Modules

import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [17]:
# Part 1 - Define a reserach question: Which Community Districts in NYC show the highest number of complaints?
# Part 2 - Import 311 data

data_311 = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')
data_311.columns

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough',
       'School Name', 'School Number', 'School Region', 'School Code',
       'School Phone Number', 'School Address', 'School City', 'School State',
       'School Zip', 'School Not Found', 'School or Citywide Complaint',
       'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location',
       'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
       'Bridge Highway Segment', 'Garage Lot Name', 'Ferry 

In [18]:
# Drop unnecessary columns

reduct_311 = data_311.drop(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Location Type', 'Incident Zip','Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type','City', 'Landmark', 'Facility Type', 'Status', 
       'Due Date', 'Resolution Description', 'Resolution Action Updated Date', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough', 'School Name', 'School Number', 
       'School Region', 'School Code', 'School Phone Number', 'School Address', 'School City', 'School State',
       'School Zip', 'School Not Found', 'School or Citywide Complaint','Vehicle Type', 'Taxi Company Borough', 
       'Taxi Pick Up Location','Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment', 
       'Garage Lot Name', 'Ferry Direction', 'Ferry Terminal Name', 'Latitude', 'Longitude', 'Location'], axis = 1)
reduct_311.head(5)

,Complaint Type,Descriptor,Community Board,Borough
0,Street Condition,Pothole,13 BROOKLYN,BROOKLYN
1,Opinion for the Mayor,HOUSING,0 Unspecified,Unspecified
2,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,18 BROOKLYN,BROOKLYN
3,Illegal Parking,Blocked Hydrant,12 BROOKLYN,BROOKLYN
4,Benefit Card Replacement,Medicaid,0 Unspecified,Unspecified


In [19]:
# Check if all Boroughs and Community Districts are represented in the Data
print (reduct_311['Community Board'].unique())
print (reduct_311['Borough'].unique())

['13 BROOKLYN' '0 Unspecified' '18 BROOKLYN' '12 BROOKLYN'
 '02 STATEN ISLAND' '08 MANHATTAN' '06 BROOKLYN' '10 BRONX' '06 MANHATTAN'
 '04 BROOKLYN' '04 QUEENS' '09 MANHATTAN' '14 BROOKLYN' '05 MANHATTAN'
 '02 MANHATTAN' '10 QUEENS' '07 QUEENS' '02 QUEENS' '11 BROOKLYN'
 '05 QUEENS' '05 BROOKLYN' '09 QUEENS' '12 MANHATTAN' '01 BROOKLYN'
 '11 MANHATTAN' '03 BRONX' '03 STATEN ISLAND' '08 BROOKLYN' '03 BROOKLYN'
 '07 MANHATTAN' '10 MANHATTAN' '04 BRONX' '01 MANHATTAN' '07 BRONX'
 '09 BRONX' '03 MANHATTAN' '13 QUEENS' '02 BRONX' '12 QUEENS' '16 BROOKLYN'
 '08 QUEENS' '04 MANHATTAN' '14 QUEENS' '10 BROOKLYN' '11 QUEENS'
 '06 BRONX' '08 BRONX' '05 BRONX' '15 BROOKLYN' '02 BROOKLYN' '12 BRONX'
 '03 QUEENS' '06 QUEENS' '01 STATEN ISLAND' '01 BRONX' '01 QUEENS'
 '17 BROOKLYN' '11 BRONX' 'Unspecified QUEENS' '09 BROOKLYN'
 'Unspecified MANHATTAN' 'Unspecified BROOKLYN' '07 BROOKLYN'
 'Unspecified BRONX' '64 MANHATTAN' '81 QUEENS' '55 BROOKLYN' '83 QUEENS'
 'Unspecified STATEN ISLAND' '28 BRONX' 

In [20]:
# How many unique values do we have? 
print (len(reduct_311['Community Board'].unique()))
print (len(reduct_311['Borough'].unique()))

77
6


In [21]:
# Deleting Invalid Data - Unspecified Values, Guidance / Code from:
# http://stackoverflow.com/questions/11350770/pandas-dataframe-select-by-partial-string
reduct_311 = reduct_311[reduct_311['Community Board'].str.contains('Unspecified') == False]
reduct_311 = reduct_311[reduct_311['Borough'].str.contains('Unspecified') == False]

In [22]:
print(len(reduct_311['Community Board'].unique()))
print(len(reduct_311['Borough'].unique()))

71
5


In [23]:
# Deleting Invalid Data - Non-Real Community Boards
reduct_311['Check'] = 'NaN'
grouby_311 = reduct_311.groupby(['Community Board'], as_index = False).count()

In [24]:
grouby_311

,Community Board,Complaint Type,Descriptor,Borough,Check
0,01 BRONX,74631,74312,74631,74631
1,01 BROOKLYN,185057,184119,185057,185057
2,01 MANHATTAN,77974,77099,77974,77974
3,01 QUEENS,171484,170936,171484,171484
4,01 STATEN ISLAND,182713,182178,182713,182713
5,02 BRONX,60257,60183,60257,60257
6,02 BROOKLYN,121022,120226,121022,121022
7,02 MANHATTAN,133860,132007,133860,133860
8,02 QUEENS,114333,113945,114333,114333
9,02 STATEN ISLAND,121132,120875,121132,121132


In [32]:
# Loop through rows, splitting Community Board values and converting the first value of the split to an interger,
# if higher than 18 (max # of community boards in any given borough), mark for deletion.
for x in range(0, len(grouby_311)):
        if int(grouby_311['Community Board'][x].split(' ')[0]) < 19:
            grouby_311['Check'][x] = 'Keep'
        else:
            grouby_311['Check'][x] = 'Delete'

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
# Check Data
grouby_311

,Community Board,Complaint Type,Descriptor,Borough,Check
0,01 BRONX,74631,74312,74631,Keep
1,01 BROOKLYN,185057,184119,185057,Keep
2,01 MANHATTAN,77974,77099,77974,Keep
3,01 QUEENS,171484,170936,171484,Keep
4,01 STATEN ISLAND,182713,182178,182713,Keep
5,02 BRONX,60257,60183,60257,Keep
6,02 BROOKLYN,121022,120226,121022,Keep
7,02 MANHATTAN,133860,132007,133860,Keep
8,02 QUEENS,114333,113945,114333,Keep
9,02 STATEN ISLAND,121132,120875,121132,Keep


In [36]:
# Remove those rows with Check = Delete
final_311 = grouby_311[grouby_311['Check'].str.contains('Delete') == False]

In [43]:
# Remove unnecessary columns
final_311 = final_311.drop(['Descriptor', 'Borough', 'Check'], axis = 1)

ValueError: labels ['Descriptor' 'Borough' 'Check'] not contained in axis

In [46]:
# Rank Complaint Count
final_311['Complaint Rank'] = final_311['Complaint Type'].rank(ascending = False)
final_311

,Community Board,Complaint Type,Complaint Rank
0,01 BRONX,74631,54.0
1,01 BROOKLYN,185057,9.0
5,02 BRONX,60257,56.0
6,02 BROOKLYN,121022,38.0
7,02 MANHATTAN,133860,30.0
8,02 QUEENS,114333,39.0
9,02 STATEN ISLAND,121132,37.0
10,03 BRONX,75134,53.0
11,03 BROOKLYN,197306,3.0
12,03 MANHATTAN,150296,19.0


In [55]:
# Part 4 - Think about the interpretation of your results?
# Load Demographic Data
cd_DemoData = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')
cd_DemoData.columns

Index(['FIPS', 'cd_id', 'Total Population',
       'Population Density (per sq. mile)', '% Total Population: Male',
       '% Total Population: 18 to 24 Years',
       '% Total Population: 25 to 34 Years',
       '% Total Population: 35 to 44 Years',
       '% Population 5 Years And Over: Speak Only English',
       '% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       '% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       '% Employed Civilian Population 16 Years And Over: Service occupations',
       '% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       '% Employed Civ

In [69]:
# Reduce DataFram
demo_Reduct = cd_DemoData[['cd_id', 'Total Population','Median household income (In 2014 Inflation Adjusted Dollars)']]
demo_Reduct

# Kept population data as a good reference / base data point, and income data as a potential comparison to 
# complaint data

,cd_id,Total Population,Median household income (In 2014 Inflation Adjusted Dollars)
0,BX08,106737,54224
1,BX12,134644,44906
2,BX10,121209,54962
3,BX11,135839,47910
4,BX03,172247,22041
5,BX06,172247,22041
6,BX07,135893,35495
7,BX05,132850,20872
8,BX04,141467,27203
9,BX09,190126,36058


In [79]:
# Clean / Format cd_id field for later merge

demo_Reduct['Borough'] = 'NaN'
demo_Reduct['CD_No'] = 'NaN'

for x in range(0, len(demo_Reduct)):
        if demo_Reduct['cd_id'][x][:2] == 'BX':
            demo_Reduct['Borough'][x] = 'BRONX'
            demo_Reduct['CD_No'][x] = demo_Reduct['cd_id'][x][-2:]
        elif demo_Reduct['cd_id'][x][:2] == 'BK':
            demo_Reduct['Borough'][x] = 'BROOKLYN'
            demo_Reduct['CD_No'][x] = demo_Reduct['cd_id'][x][-2:]
        elif demo_Reduct['cd_id'][x][:2] == 'MN':
            demo_Reduct['Borough'][x] = 'MANHATTAN'
            demo_Reduct['CD_No'][x] = demo_Reduct['cd_id'][x][-2:]
        elif demo_Reduct['cd_id'][x][:2] == 'QN':
            demo_Reduct['Borough'][x] = 'QUEENS'
            demo_Reduct['CD_No'][x] = demo_Reduct['cd_id'][x][-2:]
        elif demo_Reduct['cd_id'][x][:2] == 'SI':
            demo_Reduct['Borough'][x] = 'STATEN ISLAND'
            demo_Reduct['CD_No'][x] = demo_Reduct['cd_id'][x][-2:]



/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://panda

In [87]:
# Concatenate new columns into new, merge compatible 
demo_Reduct['cd_id'] = demo_Reduct['CD_No'] + ' ' + demo_Reduct['Borough']
demo_Reduct = demo_Reduct.drop(['Borough', 'CD_No'], axis = 1)
demo_Reduct

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,cd_id,Total Population,Median household income (In 2014 Inflation Adjusted Dollars)
0,08 BRONX,106737,54224
1,12 BRONX,134644,44906
2,10 BRONX,121209,54962
3,11 BRONX,135839,47910
4,03 BRONX,172247,22041
5,06 BRONX,172247,22041
6,07 BRONX,135893,35495
7,05 BRONX,132850,20872
8,04 BRONX,141467,27203
9,09 BRONX,190126,36058


In [90]:
# Load Infrastructure Data
df_infr = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv')
df_infr.head(5)

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [91]:
# View Columns
df_infr.columns

Index(['FIPS', 'Geographic Identifier', 'Qualifying Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Househol

Index(['FIPS', 'Geographic Identifier', 'Qualifying Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Househol